In [1]:
import sys 
sys.path.append('..')

In [5]:
from datetime import datetime as dt

import numpy as np
import pandas as pd

from preprocessing import Preprocessing
from sklearn.model_selection import StratifiedKFold, GridSearchCV


from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
#from lightgbm import LGBMClassifier

In [7]:
col_drop = ['CustomerId','Surname']
types = {
    'HasCrCard':bool,
    'IsActiveMember':bool
}
train = pd.read_csv('../data/train.csv', index_col='id', dtype=types).drop(columns=col_drop)
train.drop_duplicates(inplace=True)

In [8]:
X = train.drop(columns='Exited')
y = train.Exited

In [9]:
prepro = Preprocessing()
X_scld = prepro.fit_transform(X).astype(float)

In [10]:
skf = StratifiedKFold(n_splits=4)

In [14]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [15]:
params = {
    "booster": ['gbtree', 'gblinear', 'dart'],
    "eta": [0.01, 0.3, 0.5, 0.8],
    "gamma": [0, 1, 2, 5, 10],
    "max_depth": [3, 6, 12, 24],
    "min_child_weight": [1, 2, 3],
    "sample_method": ['uniform','gradient_based'],
    "max_leaves": [0, 1, 2],
    "n_estimators": [50, 100, 200]
}

grid = RandomizedSearchCV(
    XGBClassifier(),
    params,
    cv=skf,
    scoring='roc_auc',
    n_jobs = -1,
    verbose=10,
    n_iter=50,
)

In [16]:
grid.fit(X_scld,y)

Fitting 4 folds for each of 50 candidates, totalling 200 fits


c:\Users\Bamdantag\miniconda3\envs\brandnew\Lib\site-packages\xgboost\core.py:160: UserWarning: [12:00:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "sample_method" } are not used.

  warnings.warn(smsg, UserWarning)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           impor...
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'booster': ['gbtree', 'gblinear',
                                                    'dart'],
                                        'eta': [0.01, 0.3, 0.5, 0.8],
                                        'gamma': [0, 1, 2, 5, 10],
                                        'max_depth': [3, 6, 12, 24],
                                        'max_leaves': [0, 1, 2],
                                        'min_child_weight': [1, 2, 3],
                                        'n_estimators': [50, 100, 200],
                                        'sample_method': ['uniform',
                                                          'gradient_based']},
                   scoring='roc_auc', verbose=10)

In [17]:
grid.best_params_

{'sample_method': 'uniform',
 'n_estimators': 100,
 'min_child_weight': 2,
 'max_leaves': 0,
 'max_depth': 3,
 'gamma': 0,
 'eta': 0.5,
 'booster': 'gbtree'}

In [18]:
grid.best_score_

0.8885222673901509

In [50]:
params_specific = {
    'n_estimators': np.arange(75,151,3),
    'eta': np.arange(0.45,0.56, 0.01),
    #"min_child_weight": np.arange(1.5,2.6, 0.1),
    #'max_depth': [1,2,3,4,5]

}

In [51]:
grid = RandomizedSearchCV(
    XGBClassifier(
    sample_method='uniform',
    min_child_weight=2,
    max_leaves=0,
    max_depth=3,
    gamma=0,
    booster='gbtree',
    n_estimators=75,
    eta=0.47),
    params_specific,
    cv=skf,
    scoring='roc_auc',
    n_jobs = -1,
    verbose=10,
    n_iter=100
)

In [53]:
grid.best_params_

{'n_estimators': 75, 'eta': 0.47000000000000003}

In [54]:
grid.best_score_

0.8888722360385647

In [52]:
grid.fit(X_scld,y)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


c:\Users\Bamdantag\miniconda3\envs\brandnew\Lib\site-packages\xgboost\core.py:160: UserWarning: [12:23:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "sample_method" } are not used.

  warnings.warn(smsg, UserWarning)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
                   estimator=XGBClassifier(base_score=None, booster='gbtree',
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=0, grow_policy=None,
                                           impo...
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'eta': array([0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56]),
                                        'n_estimators': array([ 75,  78,  81,  84,  87,  90,  93,  96,  99, 102, 105, 108, 111,
       114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150])},
                   scoring='roc_auc', verbose=10)

In [55]:
optimised = XGBClassifier(
    sample_method='uniform',
    min_child_weight=2,
    max_leaves=0,
    max_depth=3,
    gamma=0,
    booster='gbtree',
    n_estimators=75,
    eta=0.47)

In [56]:
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_eta,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
17,1.138464,0.075665,0.074260,0.002573,75,0.47,"{'n_estimators': 75, 'eta': 0.47000000000000003}",0.890579,0.888176,0.888301,0.888432,0.888872,0.000990,1
3,0.950459,0.058928,0.072044,0.004393,78,0.47,"{'n_estimators': 78, 'eta': 0.47000000000000003}",0.890537,0.888185,0.888346,0.888400,0.888867,0.000967,2
53,1.042064,0.037283,0.070795,0.003542,81,0.47,"{'n_estimators': 81, 'eta': 0.47000000000000003}",0.890577,0.888083,0.888349,0.888345,0.888839,0.001009,3
76,1.072447,0.039933,0.074388,0.005947,84,0.47,"{'n_estimators': 84, 'eta': 0.47000000000000003}",0.890590,0.888060,0.888352,0.888317,0.888830,0.001022,4
39,1.323425,0.072568,0.093577,0.004796,111,0.47,"{'n_estimators': 111, 'eta': 0.47000000000000003}",0.890486,0.888287,0.888429,0.888096,0.888824,0.000966,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1.238438,0.060862,0.085474,0.003736,111,0.54,"{'n_estimators': 111, 'eta': 0.54}",0.889658,0.886815,0.887482,0.887621,0.887894,0.001063,96
94,1.595803,0.207533,0.095567,0.001715,120,0.54,"{'n_estimators': 120, 'eta': 0.54}",0.889588,0.886893,0.887415,0.887635,0.887883,0.001021,97
11,1.534147,0.089841,0.092574,0.002621,117,0.54,"{'n_estimators': 117, 'eta': 0.54}",0.889552,0.886858,0.887431,0.887680,0.887880,0.001010,98
50,1.552141,0.036358,0.109432,0.002168,150,0.56,"{'n_estimators': 150, 'eta': 0.56}",0.889785,0.887173,0.887251,0.887274,0.887871,0.001106,99
